In [1]:
import pandas as pd
import os
import sqlite3
from datetime import datetime
from time import time
import json
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
def cal_duration(launched,deadline):
    diff = deadline - launched
    duration = diff/86400
    return round(duration)

def find_pledged(Pledged):
    value_list = list(filter(str.isdigit, Pledged))
    join_list = "".join(value_list)
    return float(join_list)

def search_key(search_item,Categories):
    for Category, items in Categories.items():
        if search_item in items: return Category

In [3]:
db_FILE = 'Kickstarter.db'
con = sqlite3.connect( db_FILE )
cur = con.cursor()

In [4]:
sql = '''
select p.Project_Id, p.Project_launched_at, p.Project_deadline, p.Project_goal,p.Project_category,\
u.Date ,u.Days_to_go, u.Pledged, u.Options_number,u.Title_count,u.Blurb_count,u.Content_text_length,\
u.Content_img_count,u.Faq_count,u.Update_count,u.Comment_count,u.Backers,v.Video_name, p.Cancel_check,p.Project_title,p.Project_blurb,\
c.Backed_count, c.Created_count 

    FROM Video as v

    Left JOIN Project as p
        ON v.Project_Id = p.Project_Id
        
    Left JOIN Upd as u
        ON p.Project_Id = u.Project_Id

    Left JOIN Creator as c
        ON u.Creator_Id = c.Creator_Id

        
'''
cur.execute(sql)
rows = cur.fetchall()

In [5]:
col_name = [i[0] for i in cur.description]

In [6]:
#current_time = time()
#7/13 unix time
current_time = 1563022800

In [7]:
df = pd.DataFrame(rows)
df.columns = col_name

In [8]:
print('Raw data : {0}'.format(df.shape))

# 1차 filtering (deadline) 현재 시간 기준으로
df = df.loc[lambda df : df.Project_deadline < current_time]
print('1차 filtering : {0}'.format(df.shape))

# 2차 filtering (luanched_date)
luanched_date = 1558483200 #0523
df = df.loc[lambda df : df.Project_launched_at > luanched_date]
print('2차 filtering : {0}'.format(df.shape))

# 3차 filtering (nan값 제거)
df = df[df['Pledged'].notnull()]
print('3차 filtering : {0}'.format(df.shape) )

# unique project 추출 (마지막 update 값으로)
unique_df = df.drop_duplicates(subset=['Project_Id'],keep='last')

# 4차 filtering (crawling이 잘된 Project diff가 5 미만)
Project_launched_at = unique_df['Project_launched_at'].values
Project_deadline = unique_df['Project_deadline'].values
duration = [cal_duration(launched,deadline) for launched, deadline in zip(Project_launched_at,Project_deadline)]

# crawl num
project_id_list = unique_df['Project_Id'].unique().tolist()
num_list = []
for project_id in project_id_list:
    find_df = df[df['Project_Id']==project_id]
    num_crawl = find_df.shape[0]
    num_list.append(num_crawl)
    
unique_df['num_crawl'] = num_list
unique_df['duration'] = duration

# 전체 5일 미만 Crawling잘된 project filtering
unique_df['diff'] = unique_df['duration'] - unique_df['num_crawl']
unique_df = unique_df[unique_df['diff'] < 5]

print('4차 filtering : {0}'.format(unique_df.shape))

# 5차 filtering (최소 7일이상 funding이 진행된 project)
unique_df = unique_df[unique_df['num_crawl'] > 6]

print('5차 filtering : {0}'.format(unique_df.shape))

Raw data : (113628, 23)
1차 filtering : (70125, 23)
2차 filtering : (37123, 23)
3차 filtering : (37119, 23)
4차 filtering : (1333, 26)
5차 filtering : (1315, 26)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### success-fail 나누기

In [9]:
project_id_list = unique_df['Project_Id'].unique().tolist()
success_list = []

for project_id in project_id_list:
    find_df = df[df['Project_Id']==project_id]
    last_element = find_df.iloc[-1]
    Project_goal = last_element['Project_goal']
    Pledged = find_pledged(last_element['Pledged'])
    success = 0
    
    if Pledged > Project_goal:
        success = 1
        
    success_list.append(success)

unique_df['Success'] = success_list

### Backers & Ratio 구하기

In [10]:
df['Pledged'] = df['Pledged'].apply(lambda x : find_pledged(x))
df['Backers'] = df['Backers'].apply(lambda x : find_pledged(x))
unique_df['Pledged'] = unique_df['Pledged'].apply(lambda x : find_pledged(x))


project_id_list = unique_df['Project_Id'].unique().tolist()
pledged_ratio = [[],[],[]]
backers_days = [[],[],[]]

for project_id in project_id_list:
    find_df = df[df['Project_Id'] == project_id].iloc[0:3]
    
    for i in range(0,3):
        element = find_df.iloc[i]
        ## backers count
        backers = element['Backers']
        backers_days[i].append(backers)
        ## pledged
        project_goal = element['Project_goal']
        pledged = element['Pledged']

        ratio = (pledged / project_goal) * 100
        pledged_ratio[i].append(ratio)

In [11]:
unique_df['backer1'] = backers_days[0]
unique_df['backer2'] = backers_days[1]
unique_df['backer3'] = backers_days[2]

In [12]:
unique_df['ratio1'] = pledged_ratio[0]
unique_df['ratio2'] = pledged_ratio[1]
unique_df['ratio3'] = pledged_ratio[2]

### Category 분류

### Top category 분류

In [13]:
with open('dict_cateogry.json') as data_file:    
    TopDict = json.load(data_file)

In [14]:
json_data=open('dict_cateogry.json').read()
Categories = json.loads(json_data)
unique_df['Top_category'] = unique_df['Project_category'].apply(lambda x : search_key(x,Categories))
unique_df['Top_category'] = unique_df['Project_category'].apply(lambda x : search_key(x,Categories))

### Super Category 분류 

In [15]:
super_category = {
    'Arts' : ['Art','Dance','Photography','Theater'],
    'Comics & Illustartion' : ['Comics','Illustration'],
    'Design & Tech' : ['Design','Technology'],
    'Film' : ['Film & Video'],
    'Food & Craft' : ['Crafts','Fashion','Food'],
    'Games' : ['Games'],
    'Music' : ['Music'],
    'Publishing' : ['Journalism','Publishing']
}

In [16]:
unique_df['super_category'] = unique_df['Top_category'].apply(lambda x : search_key(x,super_category))
unique_df['super_category'] = unique_df['Top_category'].apply(lambda x : search_key(x,super_category))

### Dataframe 저장

In [17]:
unique_df.to_csv('analysis_data.csv',index=False,encoding='utf-16')
df.to_csv('analysis_update_data.csv',index=False,encoding='utf-16')

### Survey용 Dataframe
- Project_Id
- Project_title
- Project_blurb
- super_category
- Success
- (Video_URL)

In [18]:
survey_df = unique_df[['Project_Id','Project_title','Project_blurb','super_category','Success']]

### Survey용 Dataframe 저장

In [19]:
survey_df.to_csv('survey_df.csv', index=False, encoding='utf-16')